In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# ==============================================================================
# PART 1: PROCESS ALL 30 DISTRICT WEATHER FILES
# ==============================================================================
district_filenames = {
    'ANUGUL':'data/anugul.csv',
    'BALANGIR':'data/balangir.csv',
    'BALESHWAR':'data/baleshwar.csv',
    'BARGARH':'data/bargarh.csv',
    'BHADRAK':'data/bhadrak.csv',
    'BOUDH':'data/boudh.csv',
    'CUTTACK':'data/cuttack.csv',
    'DEOGARH':'data/deogarh.csv',
    'DHENKANAL':'data/dhenkanal.csv',
    'GAJAPATI':'data/gajapati.csv',
    'GANJAM':'data/ganjam.csv',
    'JAGATSINGHPUR':'data/jagatsinghpur.csv',
    'JAJAPUR':'data/jajapur.csv',
    'JHARSUGUDA':'data/jharsuguda.csv',
    'KALAHANDI':'data/kalahandi.csv',
    'KANDHAMAL':'data/kandhamal.csv',
    'KENDRAPARA':'data/kendrapara.csv',
    'KENDUJHAR':'data/kendujhar.csv',
    'KHORDHA':'data/khordha.csv',
    'KORAPUT':'data/koraput.csv',
    'MALKANGIRI':'data/malkangiri.csv',
    'MAYURBHANJ':'data/mayurbhanj.csv',
    'NABARANGPUR':'data/nabarangpur.csv',
    'NAYAGARH':'data/nayagarh.csv',
    'NUAPADA':'data/nuapada.csv',
    'PURI': 'data/puri.csv',
    'RAYAGADA':'data/rayagada.csv',
    'SAMBALPUR':'data/sambalpur.csv',
    'SONEPUR':'data/sonepur.csv',
    'SUNDARGARH':'data/sundargarh.csv'
}

# An empty list to store the processed data for each district
all_weather_data = []

def get_season(month):
    if 6 <= month <= 10: return 'Kharif'
    elif month in [11, 12, 1, 2, 3]: return 'Rabi'
    else: return 'Summer'

print("Starting to process weather files...")
for district_name, file_name in district_filenames.items():
    try:
        temp_df = pd.read_csv(file_name, skiprows=13)
        temp_df['DATE'] = pd.to_datetime(temp_df['YEAR'].astype(str) + temp_df['DOY'].astype(str), format='%Y%j')
        temp_df['Month'] = temp_df['DATE'].dt.month
        temp_df['Season'] = temp_df['Month'].apply(get_season)
        seasonal_data = temp_df.groupby(['YEAR', 'Season']).agg(avg_temp=('T2M', 'mean'),max_temp=('T2M_MAX', 'max'),min_temp=('T2M_MIN', 'min'),total_rainfall=('PRECTOTCORR', 'sum')).reset_index()
        seasonal_data.rename(columns={'YEAR': 'Year'}, inplace=True)
        seasonal_data['District_Name'] = district_name
        all_weather_data.append(seasonal_data)
    except FileNotFoundError:
        print(f"  ❌ ERROR: File not found for {district_name}: '{file_name}'.")
final_weather_df = pd.concat(all_weather_data, ignore_index=True)
print("\n✅ All weather files have been processed and combined!")


Starting to process weather files...

✅ All weather files have been processed and combined!


In [3]:
# ==============================================================================
# PART 2: LOAD AND PREPARE CROP DATA
# ==============================================================================

print("\nLoading and cleaning the main crop production data (APY.csv)...")
df_crop = pd.read_csv('data/APY.csv')
df_crop.columns = df_crop.columns.str.strip()
df_odisha = df_crop[df_crop['State'] == 'Odisha'].copy()
df_odisha.rename(columns={'Crop_Year': 'Year', 'District': 'District_Name'}, inplace=True)
print("✅ Crop data loaded.")


Loading and cleaning the main crop production data (APY.csv)...
✅ Crop data loaded.


In [4]:
# ==============================================================================
# PART 3: DEBUG AND MERGE
# ==============================================================================

print("\n--- DEBUGGING: CHECKING KEYS BEFORE MERGE ---")

# Standardize the keys
df_odisha['Season_Clean'] = df_odisha['Season'].str.strip()
df_odisha['District_Name_Clean'] = df_odisha['District_Name'].str.upper()

# Print the unique values from both DataFrames
print("\n1. Unique districts in Crop Data:")
print(sorted(df_odisha['District_Name_Clean'].unique()))

print("\n2. Unique districts in Weather Data:")
print(sorted(final_weather_df['District_Name'].unique()))

print("\n3. Unique seasons in Crop Data (after cleaning spaces):")
print(df_odisha['Season_Clean'].unique())

print("\n4. Unique seasons in Weather Data:")
print(final_weather_df['Season'].unique())

print("\n--- END DEBUGGING ---")


# Perform the merge using the CLEANED columns
master_df = pd.merge(df_odisha, final_weather_df,
                     left_on=['District_Name_Clean', 'Year', 'Season_Clean'],
                     right_on=['District_Name', 'Year', 'Season'],
                     how='left')

# Drop helper columns
master_df = master_df.drop(columns=['District_Name_Clean', 'Season_Clean'])

master_df.dropna(subset=['avg_temp'], inplace=True)
print("\n✅ Merge complete!")
print("\n--- MASTER DATASET READY FOR MODEL TRAINING ---")
print(master_df.head())
print(f"\nYour final dataset has {len(master_df)} rows.")


--- DEBUGGING: CHECKING KEYS BEFORE MERGE ---

1. Unique districts in Crop Data:
['ANUGUL', 'BALANGIR', 'BALESHWAR', 'BARGARH', 'BHADRAK', 'BOUDH', 'CUTTACK', 'DEOGARH', 'DHENKANAL', 'GAJAPATI', 'GANJAM', 'JAGATSINGHAPUR', 'JAJAPUR', 'JHARSUGUDA', 'KALAHANDI', 'KANDHAMAL', 'KENDRAPARA', 'KENDUJHAR', 'KHORDHA', 'KORAPUT', 'MALKANGIRI', 'MAYURBHANJ', 'NABARANGPUR', 'NAYAGARH', 'NUAPADA', 'PURI', 'RAYAGADA', 'SAMBALPUR', 'SONEPUR', 'SUNDARGARH']

2. Unique districts in Weather Data:
['ANUGUL', 'BALANGIR', 'BALESHWAR', 'BARGARH', 'BHADRAK', 'BOUDH', 'CUTTACK', 'DEOGARH', 'DHENKANAL', 'GAJAPATI', 'GANJAM', 'JAGATSINGHPUR', 'JAJAPUR', 'JHARSUGUDA', 'KALAHANDI', 'KANDHAMAL', 'KENDRAPARA', 'KENDUJHAR', 'KHORDHA', 'KORAPUT', 'MALKANGIRI', 'MAYURBHANJ', 'NABARANGPUR', 'NAYAGARH', 'NUAPADA', 'PURI', 'RAYAGADA', 'SAMBALPUR', 'SONEPUR', 'SUNDARGARH']

3. Unique seasons in Crop Data (after cleaning spaces):
['Autumn' 'Summer' 'Winter' 'Kharif' 'Rabi' 'Whole Year']

4. Unique seasons in Weather Data

In [5]:
# Check the merged data and export to CSV
print("Master DataFrame Info:")
print(f"Shape: {master_df.shape}")
print(f"Columns: {list(master_df.columns)}")
print("\nFirst few rows:")
print(master_df.head())

# Export the merged data to CSV
output_file = "merged_apy_data.csv"
master_df.to_csv(output_file, index=False)
print(f"\nMerged data exported to: {output_file}")
print(f"File saved with {len(master_df)} rows and {len(master_df.columns)} columns.")

Master DataFrame Info:
Shape: (7227, 14)
Columns: ['State', 'District_Name_x', 'Crop', 'Year', 'Season_x', 'Area', 'Production', 'Yield', 'Season_y', 'avg_temp', 'max_temp', 'min_temp', 'total_rainfall', 'District_Name_y']

First few rows:
    State District_Name_x       Crop  Year     Season_x    Area  Production  \
1  Odisha          ANUGUL  Arhar/Tur  1997  Summer        469.0       115.0   
3  Odisha          ANUGUL  Arhar/Tur  1999  Kharif       7960.0      5010.0   
4  Odisha          ANUGUL  Arhar/Tur  2000  Kharif       8930.0      6430.0   
5  Odisha          ANUGUL  Arhar/Tur  2002  Kharif       8730.0      6050.0   
6  Odisha          ANUGUL  Arhar/Tur  2003  Kharif       9500.0      6500.0   

   Yield Season_y   avg_temp  max_temp  min_temp  total_rainfall  \
1   0.25   Summer  31.696393     45.56     18.99          135.95   
3   0.63   Kharif  26.761830     41.06     16.73         1217.58   
4   0.72   Kharif  26.927124     37.95     15.42          856.98   
5   0.69   Kh